# Use examples of [premise](https://github.com/romainsacchi/premise)

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `premise` to create life cycle invneotries databases for prospective environmental impact assessment.

This library extract useful information from IAM model output files (such as those of REMIND or IMAGE) and aligns inventories in the ecoinvent database accordingly.

With version 0.2.0, the following transformation are available:

* `update_electricity_to_iam_data()`: create regional electricity markets and adjust efficiency of power plants
* `update_cement_to_iam_data()`: creates regional markets for clinker production and adjust clinker production efficiency
* `update_steel_to_iam_data()`: creates regional markets for steel and adjust steel production efficiency
* `update_cars()`: updates car inventories and fleet average car transport activities
* `update_trucks()`: updates truck inventories and fleet average trucks transport activities
* `update_solar_PV()`: updates efficeincy of solar PV modules

Additional documentation on the moethodology is available [here](https://premise.readthedocs.io/en/latest/introduction.html).

## Requirements

* Pyhton 3.7 or above
* a user license for ecoinvent v.3

# Use case with [brightway2](https://brightway.dev/)

`brightway2` is an open source LCA framework for Python.
To use `premise` from `brightway2`, it requires that you have a open `brightway2` project with abiosphere as well as an
ecoinvent v.3 database registered in that project.

In [1]:
from premise import *
import brightway2 as bw

In [2]:
bw.projects.set_current("import_36_for_carculator")

In [3]:
bw.databases

Databases dictionary with 6 object(s):
	additional_biosphere
	biosphere3
	carculator db
	ecoinvent 3.5 cutoff
	ecoinvent 3.6 cutoff
	ecoinvent 3.7 cutoff

### List of available scenarios

Some scenarios come installed with the library.
They are stored in `data/iam_ouput_files` from the root directory.
They are all within the SSP2 pathway (middle of the road).

With REMIND:
* "SSP2-Base"
* "SSP2-NPi"
* "SSP2-NDC"
* "SSP2-PkBudg1300"
* "SSP2-PkBudg1100"
* "SSP2-PkBudg900"

With IMAGE:
* "SSP2-Base"


### Database creation from default scenarios

To create a scenario using REMIND's SSP2 Base pathway, from ecoinvent 3.6 for the year 2028, one would execute the following cell. This leads to the extraction of the database, some cleanup as well as importing a few additional inventories.

In [4]:
ndb = NewDatabase(
            scenarios=[
                {"model":"remind", "pathway":"SSP2-Base", "year":2028}
            ],
            source_db="ecoinvent 3.6 cutoff", # <-- name of the database in the BW2 project. Must be a string.
            source_version="3.6", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.7.1". Must be a string.
    )


////////////////////// EXTRACTING SOURCE DATABASE ///////////////////////
Getting activity data


  0%|                                               | 0/615644 [00:00<?, ?it/s]

Adding exchange data to activities


100%|███████████████████████████████| 615644/615644 [00:36<00:00, 17030.34it/s]


Filling out exchange data


100%|██████████████████████████████████| 18121/18121 [00:02<00:00, 6594.61it/s]


Set missing location of datasets to global scope.
Set missing location of production exchanges to scope of dataset.
Correct missing location of technosphere exchanges.
Remove empty exchanges.

/////////////////// IMPORTING ADDITIONAL INVENTORIES ////////////////////
Importing necessary inventories...

Done!



or here with ecoinvent 3.7.1

In [5]:
ndb = NewDatabase(
        scenarios=[
                {"model":"remind", "pathway":"SSP2-Base", "year":2028}
            ],
        source_db="ecoinvent 3.7 cutoff", # <-- this is NEW.
        source_version="3.7.1", # <-- this is NEW
    )


////////////////////// EXTRACTING SOURCE DATABASE ///////////////////////
Getting activity data


100%|█████████████████████████████████| 19128/19128 [00:01<00:00, 14518.67it/s]


Adding exchange data to activities


100%|███████████████████████████████| 621717/621717 [00:50<00:00, 12265.49it/s]


Filling out exchange data


100%|██████████████████████████████████| 19128/19128 [00:04<00:00, 3881.96it/s]


Set missing location of datasets to global scope.
Set missing location of production exchanges to scope of dataset.
Correct missing location of technosphere exchanges.
Remove empty exchanges.

/////////////////// IMPORTING ADDITIONAL INVENTORIES ////////////////////
Importing necessary inventories...

Done!



If you want to create multiple databases at once, just populate the `scenarios` list.

In [6]:
ndb = NewDatabase(
            scenarios=[
                {"model":"remind", "pathway":"SSP2-Base", "year":2028},
                {"model":"remind", "pathway":"SSP2-NPi", "year":2032}
            ],
            source_db="ecoinvent 3.5 cutoff", # <-- name of the database. Must be a string.
            source_version="3.5", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.7.1"
                 )


////////////////////// EXTRACTING SOURCE DATABASE ///////////////////////
Getting activity data


  0%|                                               | 0/544735 [00:00<?, ?it/s]

Adding exchange data to activities


100%|███████████████████████████████| 544735/544735 [00:48<00:00, 11121.79it/s]


Filling out exchange data


100%|██████████████████████████████████| 16022/16022 [00:02<00:00, 5730.24it/s]


Set missing location of datasets to global scope.
Set missing location of production exchanges to scope of dataset.
Correct missing location of technosphere exchanges.
Remove empty exchanges.

/////////////////// IMPORTING ADDITIONAL INVENTORIES ////////////////////
Importing necessary inventories...

Done!



When the database is loaded and the additional inventories imported, you can apply a transformation function.
For example here, we adjust the efficiency of the solar PVs to the two scenarios we have loaded.
We go more in details later.

In [7]:
ndb.update_solar_PV()


/////////////////// SOLAR PV ////////////////////
Update efficiency of solar PVs.

Update efficiency of solar PVs.



And then, we register these two databases back into brightway2.

In [ ]:
ndb.write_db_to_brightway()

### Database creation from non-default scenarios

If you have some specific IAM scenarios (one that is not included in `premise`) you would like to build a database from, you can specify the directory to those.

**Important remark**: your scenario file must begin with "remind_" or "image_".

In [4]:
ndb = NewDatabase(
    scenarios = [{"model":"remind", "pathway":"my_special_scenario", "year":2028, "filepath":r"C:\Users\sacchi_r\Downloads\REMIND"}],        
    source_db="ecoinvent 3.6 cutoff", # <-- name of the database
    source_version=3.6, # <-- version of ecoinvent
                 )


////////////////////// EXTRACTING SOURCE DATABASE ///////////////////////
Getting activity data


  0%|                                               | 0/615644 [00:00<?, ?it/s]

Adding exchange data to activities


100%|███████████████████████████████| 615644/615644 [00:35<00:00, 17386.04it/s]


Filling out exchange data


100%|██████████████████████████████████| 18121/18121 [00:02<00:00, 6153.55it/s]


Set missing location of datasets to global scope.
Set missing location of production exchanges to scope of dataset.
Correct missing location of technosphere exchanges.
Remove empty exchanges.

/////////////////// IMPORTING ADDITIONAL INVENTORIES ////////////////////
Importing necessary inventories...

Done!



# Use case with ecospold2

The source database does not have to be from a brightway2 project.
It can be directly extracted from the bunch of ecospold2 files one gets when downloaded from the [ecoinvent website](https://ecoinvent.org).

For this, one needs to specify the argument `source_db = "ecospold"` as well as `source_file_path`, which is the directory leading to the ecospold files.

For example, here we combine the use of a specific (non-default) IAM scenario file with the use of ecospold2 files as data source (ecoinvent 3.5 in this case).

In [6]:
ndb = NewDatabase(
        scenarios = [
            {"model":"remind", "pathway":"my_special_scenario", "year":2028, "filepath":r"C:\Users\sacchi_r\Downloads\REMIND"}
        ],        
        source_type="ecospold", # <--- this is NEW
        source_file_path=r"C:\Users\sacchi_r\Dropbox\Public\ecoinvent 3.5_cutoff_ecoSpold02\datasets", # <-- this is NEW
        source_version=3.5,
    )


////////////////////// EXTRACTING SOURCE DATABASE ///////////////////////
Extracting XML data from 16022 datasets
Extracted 16022 datasets in 111.46 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying

# Transformation functions

These functions modify the extracted database:

* **update_electricity_to_iam_data()**: alignment of regional electricity production mixes as well as efficiencies for a number of
  electricity production technologies, including Carbon Capture and Storage technologies.
* **update_cement_to_iam_data()**: adjustment of technologies for cement production (dry, semi-dry, wet, with pre-heater or not),
  fuel efficiency of kilns, fuel mix of kilns (including biomass and waste fuels) and clinker-to-cement ratio.
* **update_steel_to_iam_data()**: adjustment of process efficiency, fuel mix and share of secondary steel in steel markets.
* **update_solar_PV()**: adjustment of solar PV panels efficiency to the year considered.

They can be applied *separately*, *consecutively* or *altogether* (using instead **.update_all()**).
They will apply to all the scenario-specific databases listed in `scenarios`.

In [7]:
ndb.update_all()


/////////////////// ELECTRICITY ////////////////////
Remove old electricity datasets
Create high voltage markets.
Create medium voltage markets.
Create low voltage markets.
Link activities to new electricity markets.
Log of deleted electricity markets saved in C:\Users\sacchi_r\Documents\GitHub\premise\premise\data\logs
Log of created electricity markets saved in C:\Users\sacchi_r\Documents\GitHub\premise\premise\data\logs
Log of changes in power plants efficiencies saved in C:\Users\sacchi_r\Documents\GitHub\premise\premise\data\logs
Rescale inventories and emissions for Biomass IGCC CCS
The current efficiency factor for the dataset electricity production, at BIGCC power plant 450MW, pre, pipeline 200km, storage 1000m has not been found.Its current efficiency will remain
Rescale inventories and emissions for Biomass CHP
Rescale inventories and emissions for Biomass IGCC
The current efficiency factor for the dataset electricity production, at BIGCC power plant 450MW, no CCS has not be

No dataset cement production, blast furnace slag 36-65% found for the REMIND region LAM
No dataset cement production, blast furnace slag 36-65% found for the REMIND region OAS
No dataset cement production, blast furnace slag 36-65% found for the REMIND region SSA
No dataset cement production, blast furnace slag 36-65% found for the REMIND region EUR
No dataset cement production, blast furnace slag 36-65% found for the REMIND region NEU
No dataset cement production, blast furnace slag 36-65% found for the REMIND region MEA
No dataset cement production, blast furnace slag 36-65% found for the REMIND region REF
No dataset cement production, blast furnace slag 36-65% found for the REMIND region CAZ
No dataset cement production, blast furnace slag 36-65% found for the REMIND region CHA
No dataset cement production, blast furnace slag 36-65% found for the REMIND region IND
No dataset cement production, blast furnace slag 36-65% found for the REMIND region JPN
No dataset cement production, bl

No dataset cement production, limestone 21-35% found for the REMIND region LAM
No dataset cement production, limestone 21-35% found for the REMIND region OAS
No dataset cement production, limestone 21-35% found for the REMIND region SSA
No dataset cement production, limestone 21-35% found for the REMIND region EUR
No dataset cement production, limestone 21-35% found for the REMIND region NEU
No dataset cement production, limestone 21-35% found for the REMIND region MEA
No dataset cement production, limestone 21-35% found for the REMIND region REF
No dataset cement production, limestone 21-35% found for the REMIND region CAZ
No dataset cement production, limestone 21-35% found for the REMIND region CHA
No dataset cement production, limestone 21-35% found for the REMIND region IND
No dataset cement production, limestone 21-35% found for the REMIND region JPN
No dataset cement production, limestone 21-35% found for the REMIND region USA
No dataset cement production, limestone 21-35% found


Create new cement market datasets
No dataset market for cement, blast furnace slag 35-70% found for the REMIND region LAM
No dataset market for cement, blast furnace slag 35-70% found for the REMIND region OAS
No dataset market for cement, blast furnace slag 35-70% found for the REMIND region SSA
No dataset market for cement, blast furnace slag 35-70% found for the REMIND region EUR
No dataset market for cement, blast furnace slag 35-70% found for the REMIND region NEU
No dataset market for cement, blast furnace slag 35-70% found for the REMIND region MEA
No dataset market for cement, blast furnace slag 35-70% found for the REMIND region REF
No dataset market for cement, blast furnace slag 35-70% found for the REMIND region CAZ
No dataset market for cement, blast furnace slag 35-70% found for the REMIND region CHA
No dataset market for cement, blast furnace slag 35-70% found for the REMIND region IND
No dataset market for cement, blast furnace slag 35-70% found for the REMIND region J

No dataset market for cement, pozzolana and fly ash 11-35% found for the REMIND region LAM
No dataset market for cement, pozzolana and fly ash 11-35% found for the REMIND region OAS
No dataset market for cement, pozzolana and fly ash 11-35% found for the REMIND region SSA
No dataset market for cement, pozzolana and fly ash 11-35% found for the REMIND region EUR
No dataset market for cement, pozzolana and fly ash 11-35% found for the REMIND region NEU
No dataset market for cement, pozzolana and fly ash 11-35% found for the REMIND region MEA
No dataset market for cement, pozzolana and fly ash 11-35% found for the REMIND region REF
No dataset market for cement, pozzolana and fly ash 11-35% found for the REMIND region CAZ
No dataset market for cement, pozzolana and fly ash 11-35% found for the REMIND region CHA
No dataset market for cement, pozzolana and fly ash 11-35% found for the REMIND region IND
No dataset market for cement, pozzolana and fly ash 11-35% found for the REMIND region JPN

No dataset market for cement, blast furnace slag 5-25% found for the REMIND region LAM
No dataset market for cement, blast furnace slag 5-25% found for the REMIND region OAS
No dataset market for cement, blast furnace slag 5-25% found for the REMIND region SSA
No dataset market for cement, blast furnace slag 5-25% found for the REMIND region EUR
No dataset market for cement, blast furnace slag 5-25% found for the REMIND region NEU
No dataset market for cement, blast furnace slag 5-25% found for the REMIND region MEA
No dataset market for cement, blast furnace slag 5-25% found for the REMIND region REF
No dataset market for cement, blast furnace slag 5-25% found for the REMIND region CAZ
No dataset market for cement, blast furnace slag 5-25% found for the REMIND region CHA
No dataset market for cement, blast furnace slag 5-25% found for the REMIND region IND
No dataset market for cement, blast furnace slag 5-25% found for the REMIND region JPN
No dataset market for cement, blast furnace

#### Additional options for database creation with cars and truck inventories

Two additional functions are available:
* **update_cars()**: creates updated inventories for fleet average passenger cars and links back to activities that consume
transport.
* **update_trucks()**: creates updated inventories for fleet average lorry trucks and links back to activities that consume
transport.


By default, inventories for passenger cars and trucks are not added upon the creation of the database.
But you can add them by adding the following keys to the scenario dictionary: `passenger cars` and `trucks`.
This will create inventories for passenger cars and trucks for each region of the IAM model.

For example, here we create car inventories in the first scenario, and both cars and trucks inventories in the
second scenario. Leaving an empty dicitonary indicates that we do not apply any filter in terms of regions and that we do not provide a fleet composition file (a default one is then used).

In [4]:
ndb = NewDatabase(
            scenarios=[
                {"model":"remind", "pathway":"SSP2-Base", "year":2028, "passenger cars":{}},
                #{"model":"remind", "pathway":"SSP2-NPi", "year":2032, "passenger cars":{}, "trucks":{}}
            ],
            source_db="ecoinvent 3.5 cutoff", # <-- name of the database. Must be a string.
            source_version="3.5", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.7.1"
                 )

No fleet composition file is provided for passenger cars.Fleet average vehicles will be built using default fleet projection.

////////////////////// EXTRACTING SOURCE DATABASE ///////////////////////
Getting activity data


  0%|                                               | 0/544735 [00:00<?, ?it/s]

Adding exchange data to activities


100%|███████████████████████████████| 544735/544735 [00:37<00:00, 14428.13it/s]


Filling out exchange data


100%|██████████████████████████████████| 16022/16022 [00:03<00:00, 4853.79it/s]


Set missing location of datasets to global scope.
Set missing location of production exchanges to scope of dataset.
Correct missing location of technosphere exchanges.
Remove empty exchanges.

/////////////////// IMPORTING ADDITIONAL INVENTORIES ////////////////////
Importing necessary inventories...

Done!



In [5]:
ndb.update_cars()


/////////////////// PASSENGER CARS ////////////////////
No REMIND region could be mapped to the following ISO alpha2 country codes: ['BV' 'HM' 'PM' 'TK']
They will be skipped.


c:\users\sacchi_r\documents\github\carculator\carculator\utils.py:430: FutureWarning: The default value of regex will change from True to False in a future version.
  df["variable"] = df["variable"].str.replace(r"\D", "").astype(int)


Fleet information for 2028 is not available. We'll use 2025 instead.


c:\users\sacchi_r\documents\github\carculator\carculator\utils.py:44: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df["Variable"] = df["Variable"].str.replace("|", "-")
C:\Users\sacchi_r\AppData\Local\Continuum\miniconda3\envs\coarse\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
C:\Users\sacchi_r\AppData\Local\Continuum\miniconda3\envs\coarse\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, ord

****************** IMPORTANT BACKGROUND PARAMETERS ******************
 * The country of use is LAM
 * Power and energy batteries produced in CN using NMC chemistry.
 * in 1996, % of renewable: 56.0%, GHG intensity per kWh: 375 g. CO2-eq.
 	 * in 1997, % of renewable: 56.0%, GHG intensity per kWh: 374 g. CO2-eq.
 	 * in 1998, % of renewable: 56.0%, GHG intensity per kWh: 372 g. CO2-eq.
 	 * in 1999, % of renewable: 56.0%, GHG intensity per kWh: 370 g. CO2-eq.
 	 * in 2000, % of renewable: 57.0%, GHG intensity per kWh: 367 g. CO2-eq.
 	 * in 2001, % of renewable: 57.0%, GHG intensity per kWh: 365 g. CO2-eq.
 	 * in 2002, % of renewable: 57.0%, GHG intensity per kWh: 362 g. CO2-eq.
 	 * in 2003, % of renewable: 57.0%, GHG intensity per kWh: 360 g. CO2-eq.
 	 * in 2004, % of renewable: 57.0%, GHG intensity per kWh: 357 g. CO2-eq.
 	 * in 2005, % of renewable: 58.0%, GHG intensity per kWh: 355 g. CO2-eq.
 	 * in 2006, % of renewable: 58.0%, GHG intensity per kWh: 352 g. CO2-eq.
 	 * in 2007

...

0% [##############################] 100% | ETA: 00:00:00 | Item ID: 2385
Total time elapsed: 00:00:21


No REMIND region could be mapped to the following ISO alpha2 country codes: ['BV' 'HM' 'PM' 'TK']
They will be skipped.


c:\users\sacchi_r\documents\github\carculator\carculator\utils.py:430: FutureWarning: The default value of regex will change from True to False in a future version.
  df["variable"] = df["variable"].str.replace(r"\D", "").astype(int)


Fleet information for 2028 is not available. We'll use 2025 instead.


ValueError: Wrong number of items passed 11, placement implies 1

If you want to limit the inventories of passenger cars and trucks to specific regions, you need to pass a dictionary instead, like so:

In [3]:
ndb = NewDatabase(
            model="remind", # <-- name of the IAM model
            scenario="SSP2-Base", # <-- name of the scenario
            year=2028, # <-- year
            source_db="ecoinvent 3.6 cutoff", # <-- name of the database
            source_version=3.6, # <-- version of ecoinvent
            add_passenger_cars = {"region":["EUR", "USA"]}, # <-- only for EU and US
            add_trucks = {"region": ["CHA"]} # <-- only for China
                 )

No fleet composition file is provided, hence fleet average vehicles inventories will not be produced.
No fleet composition file is provided, hence fleet average truck inventories will not be produced.

////////////////////// EXTRACTING SOURCE DATABASE ///////////////////////
Getting activity data


100%|█████████████████████████████████| 18121/18121 [00:00<00:00, 59768.13it/s]


Adding exchange data to activities


100%|███████████████████████████████| 615644/615644 [00:41<00:00, 14691.03it/s]


Filling out exchange data


...

0% [##############################] 100% | ETA: 00:00:00 | Item ID: 866
Total time elapsed: 00:00:00
C:\Users\sacchi_r\AppData\Local\Continuum\miniconda3\envs\coarse\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:133: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
C:\Users\sacchi_r\AppData\Local\Continuum\miniconda3\envs\coarse\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:133: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)


****************** IMPORTANT BACKGROUND PARAMETERS ******************
 * The country of use is CH
 * Energy batteries produced in CN using NMC chemistry.
 * in 2028, % of renewable: 12.0%, GHG intensity per kWh: 633 g. CO2-eq.
 * electrolysis is completed by smr - natural gas.
 	 * in 2028 _________________________________________ 0.0%
 * cng is completed by biogas - sewage sludge.
 	 * in 2028 _________________________________________ 0.0%
 * diesel is completed by biodiesel - cooking oil.
 	 * in 2028 _________________________________________ 0.0%
 * *********************************************************************


0% [##############################] 100% | ETA: 00:00:00 | Item ID: 866
Total time elapsed: 00:00:00


The following datasets to import already exist in the source database. They will not be imported
+------------------------------------------+--------------------------------+----------------------------+
|                   Name                   |       Reference product        |          Location          |
+------------------------------------------+--------------------------------+----------------------------+
| Hydrogen, gaseous, 25 bar, from dual flu |   Hydrogen, gaseous, 25 bar    |             CH             |
| Hydrogen, gaseous, 700 bar, from gasific |   Hydrogen, gaseous, 700 bar   |             CH             |
|              Essential BoP               |         Essential BoP          |            GLO             |
| hardwood forestry, oak, sustainable fore | wood chips, wet, measured as d |             DE             |
| production of 2 wt-% potassium iodide so | production of 2 wt-% potassium |            RER             |
| Fuel tank, compressed hydrogen gas, 700b |   

+------------------------------------------+--------------------------------+----------------------------+


### Fleet files

A last word about passenger cars: it is possible to pass a fleet composition file to generate fleet average inventories and limit those to specific regions (here, the European region).

**Important remark**: fleet avergae invenotries generation from fleet compoistion files works only with REMIND at the moment.

In [7]:
ndb = NewDatabase(
            model="remind",
            scenario="SSP2-Base", 
            year=2028,
            source_db="ecoinvent 3.6 cutoff",
            source_version=3.6, 
            add_passenger_cars={
                "fleet file": r"C:\Users\sacchi_r\Downloads\REMIND\Budg1100_ConvSyn_vintcomp.csv", # <-- this is NEW
                "region": ["EUR"]
        },
     )


////////////////////// EXTRACTING SOURCE DATABASE ///////////////////////
Getting activity data


100%|████████████████████████████████| 18121/18121 [00:00<00:00, 127059.33it/s]


Adding exchange data to activities


100%|███████████████████████████████| 615644/615644 [00:46<00:00, 13132.40it/s]


Filling out exchange data


...

C:\Users\sacchi_r\AppData\Local\Continuum\miniconda3\envs\coarse\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:133: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
C:\Users\sacchi_r\AppData\Local\Continuum\miniconda3\envs\coarse\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:133: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)


****************** IMPORTANT BACKGROUND PARAMETERS ******************
 * The country of use is EUR
 * Power and energy batteries produced in CN using NMC chemistry.
 * in 1996, % of renewable: 9.0%, GHG intensity per kWh: 587 g. CO2-eq.
 	 * in 1997, % of renewable: 10.0%, GHG intensity per kWh: 581 g. CO2-eq.
 	 * in 1998, % of renewable: 10.0%, GHG intensity per kWh: 576 g. CO2-eq.
 	 * in 1999, % of renewable: 10.0%, GHG intensity per kWh: 570 g. CO2-eq.
 	 * in 2000, % of renewable: 11.0%, GHG intensity per kWh: 564 g. CO2-eq.
 	 * in 2001, % of renewable: 11.0%, GHG intensity per kWh: 558 g. CO2-eq.
 	 * in 2002, % of renewable: 12.0%, GHG intensity per kWh: 552 g. CO2-eq.
 	 * in 2003, % of renewable: 12.0%, GHG intensity per kWh: 546 g. CO2-eq.
 	 * in 2004, % of renewable: 12.0%, GHG intensity per kWh: 540 g. CO2-eq.
 	 * in 2005, % of renewable: 13.0%, GHG intensity per kWh: 534 g. CO2-eq.
 	 * in 2006, % of renewable: 13.0%, GHG intensity per kWh: 528 g. CO2-eq.
 	 * in 2007,

0% [##############################] 100% | ETA: 00:00:00 | Item ID: 2822
Total time elapsed: 00:00:05


The following datasets to import already exist in the source database. They will not be imported
+------------------------------------------+--------------------------------+----------------------------+
|                   Name                   |       Reference product        |          Location          |
+------------------------------------------+--------------------------------+----------------------------+
| Hydrogen, gaseous, 25 bar, from dual flu |   Hydrogen, gaseous, 25 bar    |             CH             |
| Hydrogen, gaseous, 700 bar, from gasific |   Hydrogen, gaseous, 700 bar   |             CH             |
| hardwood forestry, oak, sustainable fore | wood chips, wet, measured as d |             DE             |
| production of 2 wt-% potassium iodide so | production of 2 wt-% potassium |            RER             |
| Hydrogen, gaseous, 700 bar, from ATR of  |   Hydrogen, gaseous, 700 bar   |            RER             |
| transport, pipeline, supercritical CO2,  | tr

+------------------------------------------+--------------------------------+----------------------------+


# Export

### As a Brightway2 database

Export the modified database to brightway2

In [ ]:
ndb.write_db_to_brightway()

### As a sparse matrix representation

Or export it as a sparse matrix representation.

This will export four files:

* "A_matrix.csv": matrix coordinates and values of shape (index of activity; index of product; value) for the technosphere
* "A_matrix_index.csv": labels for indices for A matrix of shape (name of activity, reference product, unit, location, index)
* "B_matrix.csv": matrix coordinates and values of shape (index of activity; index of biosphere flow; value) for the biosphere
* "B_matrix_index.csv": labels for indices for B matrix of shape (name of biosphere flow, main compartment, sub-compartmnet, unit, index)

As a convenience, you can specifiy a directory where to store the exported matrices.
If the directory does not exist, it will be created.
If you leave it unspecified, they will be stored in **data/matrices** in the root folder of the library.

In [7]:
ndb.write_db_to_matrices(filepath=r"C:/Users/sacchi_r/Downloads/exported_matrices")

Write new database to matrix.
Matrices saved in C:\Users\sacchi_r\Downloads\exported_matrices\remind\SSP2-Base\2028.


### As a SimaPro CSV file

In [5]:
ndb.write_db_to_simapro(filepath=r"C:/Users/sacchi_r/Downloads/exported_simapro_file")

Write Simapro import file.
Simapro CSV files saved in C:\Users\sacchi_r\Downloads\exported_simapro_file\remind\SSP2-Base\2028.
